##### import

In [6]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join

##### 전산조회 전

In [7]:
# 파일 읽기
########################################################
path_dataset = r"D:\3.자산\전산 dataset\솔림\230630"
name_dataset = "개인회생새창_20230630_2328"
previous = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\230621\개인회생새창_20230621_0921_기준데이터.xlsx", sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})
########################################################
path_raw_data = join(path_dataset, name_dataset+".xlsx")
raw_data = pd.read_excel(path_raw_data, dtype={'채무자키':str, '분납키':str, '계좌키':str}).fillna("")

# 종료 제외하기(경로에 '대성'있는지 여부 체크)##################
# 원데이터 만들 때 종결건 더 꼼꼼하게 보기 때문에 여기서는 일단 종결,환매,매각,회계파산만(원데이터와 상이한 결과일 경우 체크리스트로 빼기 위함)
if path_dataset.find("대성") < 0 : # 솔림
    raw_data = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|매각예정|회계파산|DS|환매"))]
else : # 대성
    raw_data = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|환매|파산|회계파산"))]



# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
col_order = [0,6,1,2,3,4,5,70,38,45,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206]
raw_data = raw_data.iloc[:,col_order]


# 인가/미인가
인가미인가 = []
for i, v in raw_data[["현재결과", "분납키", "총분납잔금", "현재원금"]].iterrows() :
    result, opb = "", 0
    if v.현재결과 == "인가" : result = "인가"
    elif (v.현재결과 == "개시") or (v.현재결과 == "금지") : result = "미인가"
    elif (v.현재결과=="폐지취소결정") or (v.현재결과=="폐지취소결정송달") : 
        temp = previous.query('분납키==@v.분납키')["인가/미인가"].values
        if len(temp) == 1 : result = temp[0]
        else : result = "#N/A"
    else : result = "기타"

    인가미인가.append(result)


raw_data.insert(7, "인가/미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)


# 엑셀 파일로 저장
path_result = join(path_dataset, name_dataset+"_기준데이터.xlsx")
if not os.path.exists(path_result):
    with pd.ExcelWriter(path_result, mode='w', engine='openpyxl') as writer:
        raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)
else : 
    with pd.ExcelWriter(path_result, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
        raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


##### 전산조회 할 거 작업 후
- 분납상태-[진행] & 
- 현재결과-[폐지취소결정, 폐지취소결정송달, 빈셀] & 
- 인가/미인가-[기타, #N/A] 즉 인가/미인가가 아닌 거

In [8]:
# 완성된 전체 다시 읽기
all_df = pd.read_excel(path_result, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

# raw_data["계좌키"] = raw_data.분납키.apply(str) # 계좌키는 여러건 입력된 거 때문에 애초에 object타입이다.

# OPB
all_df['반영OPB'] = np.where(all_df['인가/미인가'] == '인가', all_df["총분납잔금"] , all_df["현재원금"])

# 진행중 
ing_df = all_df[(all_df["인가/미인가"] != "기타") & (all_df["분납상태"] !="중단")]


###### 신복과 충돌하는 거 제거하기 #################
if path_dataset.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : ["20462830"], "보증인" : [] } # 20462830 중복입금건. 신복이 기간이 더 남아서 신복으로 처리하기로 함
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##################################################
if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]


# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_dataset, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    all_df.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장

In [9]:
# 개인회생 계좌 나누기

# path_dataset = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_dataset, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

split_ing_df = ing_df.copy()
split_ing_df["비고"] = ""
new_rows = []
for index, row in split_ing_df.iterrows():
    account_keys = row['계좌키'].split(',')
    first_account_key = account_keys.pop(0)
    split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

    for account_key in account_keys:
        new_row = row.copy() # 일단 모든 칼럼 복사
        new_row['계좌키'] = account_key
        new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
        new_row["비고"] = first_account_key + "와 단일건" # 비고
        new_rows.append(new_row)
        

temp = pd.DataFrame(new_rows)

result = pd.concat([split_ing_df, temp], ignore_index=True)

# 저장
writer = pd.ExcelWriter(join(path_dataset, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
writer.save()
writer.close()


C:\Users\SL\AppData\Local\Temp\ipykernel_30564\3991412458.py:30: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\SL\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
